In [ ]:
import chess.gaviota
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Überprüfen der $S_n$ Mengen

> Potentielles TODO: Noch die anderen Tabellen von Python-Chess als Referenz / Vergleich aufführen

Bei der Bestimmung der $S_n$ Mengen wurden alle möglichen Züge rückwärts durchgeführt.
Damit die Retrograde Analyse erfolgreich zum Gewinnen von Endspielsituationen angewandt werden kann, muss diese Berechnung erfolgreich gewesen sein.
Um sicherzustellen, dass die Situationen, die z. B. in $S_{10}$ zu finden sind, auch wirklich in $<=10$ Zügen beendet werden, wurden in diesem Notebook Funktionen definiert, die diesen Aspekt überprüfen. Hierfür werden zuerst die berechneten $S_n$ Mengen aus der Datei geladen.

In [ ]:
S_N_Sequence_fen, S_N_Sequence_fen_short = load_s_n_fens("S_n_seq_rook")

Die erste Funktion zur Überprüfung, ist ``fen_in_lower_sequence``. Diese überprüft für eine mitgegebene ``fen``, ob diese in einer $S_n$ Menge liegt, für die gilt: 
$$
n < sequence\_index
$$ 

Die Funktion gibt hierfür einen booleschen Wert zurück.

In [ ]:
def fen_in_lower_sequence(fen, sequence_index, s_n_short):
    for s in s_n_short[:sequence_index]:
        if fen in s:
            return True
    return False

Eine weitere Überprüfung liegt darin, dass keine der Schachbretter in eine zu niedrige $S_n$ Menge eingeordnet wurde. Dies wird mit der Funktion ``every_move_of_sequence_in_lower`` umgesetzt. Hierfür erhält sie eine ``sequence_index``, die überprüft werden soll. Für jede Spielsituation wird Folgendes überprüft:

$$
\forall board \in S_n : move \in board.legal\_moves \implies board.push(move) \in S_m \land m<n
$$

Als Ergebnis gibt die Funktion eine boolesche Variable zurück, die ``True`` zurückgibt, falls die Bedingung erfüllt ist, ansonsten ``False``.

In [ ]:
def every_move_of_sequence_in_lower(sequence_index, s_n_fen, s_n_short):
    print("Checking S_" + str(sequence_index) + "...")
    sequence_fen = s_n_fen[sequence_index]
    for fen in sequence_fen:
        cur_board = chess.Board(fen)
        if not cur_board.turn:
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if not fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    print("Fen: " + cur_short + " not in lower S")
                    return False
                cur_board.pop()
            return True
        else:
            in_lower = False
            for move in cur_board.legal_moves:
                cur_board.push(move)
                cur_short = get_board_and_turn(cur_board.fen())
                if fen_in_lower_sequence(cur_short, sequence_index, s_n_short):
                    in_lower = True
                cur_board.pop()
            return in_lower


Die folgende Zelle führt diesen Test durch.

In [ ]:
for i in range(len(S_N_Sequence_fen)):
    every_move_of_sequence_in_lower(i, S_N_Sequence_fen, S_N_Sequence_fen_short)

## Syzygy

Zusätzlich zum rechnerischen Überprüfen der Ergebnisse kann eine andere Methode zum Verifizieren verwendet werden. 
Schach-Endspieldatenbanken sind keine neue Erfindung und wurden bereits von anderen Forschern entwickelt.
Ein lang anhaltendes Projekt, welches gegenwärtig Datenbanken für Situationen mit bis zu 7 Figuren auf dem Spielfeld anbietet ist [Syzygy](https://syzygy-tables.info/).
Das Projekt hält zwei Datentypen für alle Spielsituationen vor:
WDL Daten und DTZ Daten.

WDL steht für Win / Draw / Loss und gibt dem Nutzer eine Information über den Wert einer Spielsituation. 
Eine Anfrage an die Datenbank mit einer Situation wird mit einem der folgenden Werte beantwortet: -2, -1, 0, 1, 2.
Positive Werte implizieren, dass bei perfektem Spiel der aktuelle Spieler gewinnt, negative Werte bedeuten, dass der aktuelle Spieler verliert.
Die 0 bedeutet, dass das Spiel (wenn beide Seiten perfekt spielen) in einem Unentschieden endet.
Eine Zwei ist ein sicherer Sieg / Verlust, während eine Eins in einem Gewinn oder unentschieden mittels der 50-Zug Regel enden kann.

Die Interessantere der Dateien ist die DTZ-Datei.
DTZ steht für Distance to Zero. Die DTZ Tabelle enthält Werte von -100 bis 100. 
Positiv, Negativ und Null kann genau wie WDL interpretiert werden.
Die Zahlen von -100 bis -1 und 1 bis 100 geben die Anzahl der Halbzüge bis zu einem Gewinn (oder Reset der 50-Züge Regel) an.
Stetiges verringern einer positiven DTZ führt also zu einem Gewinn.

Die DTZ-Zahl einer Spielsituation kann mit dem $n$ verglichen werden, in welcher Menge $S_n$ diese Situation in der `.chessAI` eingeordnet wurde.
Stimmen diese Zahlen überein, war die Berechnung korrekt.

*Hinweis: Je nach Ausführung der Syzygy-Tabellen werden halbe oder ganze Züge gespeichert, es muss daher beim Vergleich eine Toleranz von einem $n$ akzeptiert werden.*    

Die Funktion `compare_with_syzygy()` führt diesen Vergleich durch und gibt die Anzahl falsch eingeordneter Situationen zurück. 

> TODO: Parameter + Rückgabewert angeben

In [ ]:
def compare_with_syzygy(syzygy, s_n_sequence):
    counter = 0
    
    for n in range(len(s_n_sequence)):
        count = 0
        for int in s_n_sequence[n]:
            chess_board = to_board(int, PIECE_LIST)
            if n != abs(syzygy.probe_dtz(chess_board)) != n + 1:
                count += 1
        print(f"S{n}: Syzygy believes {count} of {len(s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        counter += count
    print(f"Syzygy believes {counter} Situations are wrongly placed in the sequence.")
    return count

Die Syzygy-Dateien müssen sich entweder im Ordner `./syzygy` befinden oder der Pfad angepasst werden.

> TODO: Wenn das in der globalen Schreibweise definiert wird, wieso dann überhaupt übergeben?
> TODO: Zumindest einen Teil der Tabellen in Github bereitstellen

In [ ]:
SYZYGY = chess.syzygy.Tablebase()
SYZYGY.add_directory("./syzygy")

In [ ]:
compare_with_syzygy(SYZYGY, S_N_SEQ)


## Gaviota


In [ ]:
SWAPS = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
        "diagonal" : {x : ((x >> 3) | (x << 3)) & 63 for x in range(64)},
        "anti_diagonal" : {x : (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
    }

In [ ]:
# Mirroring mit 7 Bit Darstellung
def mirror_board(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1

    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result

In [ ]:
def mirror_all_directions(board_int):
    result = set()
    for name, swap in SWAPS.items():
        result.add(mirror_board(board_int, swap))
    return result

In [ ]:
def gen_all_integers(s_n_integers):
    result = []
    for sequence in s_n_integers:
        int_set = set()
        for int_board in sequence:
            int_set.add(int_board)
            int_set |= mirror_all_directions(int_board)
        result.append(int_set)
    return result

In [ ]:
def compare_with_gaviota(gaviota, s_n_sequence):
    counter = 0
    
    for n in range(len(s_n_sequence)):
        count = 0
        for int in s_n_sequence[n]:
            chess_board = to_board(int, PIECE_LIST)
            if n != abs(gaviota.probe_dtm(chess_board)):
                count += 1
        print(f"S{n}: Gaviota believes {count} of {len(s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        counter += count
    print(f"Gaviota believes {counter} Situations are wrongly placed in the sequence.")
    return count


In [ ]:
PIECE_LIST, S_N_SEQ = load_data("two_rooks_27_05")
S_N_SEQ = gen_all_integers(S_N_SEQ)
gaviota = chess.gaviota.open_tablebase("./gaviota")
compare_with_gaviota(gaviota, S_N_SEQ)

## Matching Rate

In [ ]:
def matchrate_for_player(turn):
    # TODO: Hier muss noch irgendwo eine Berücksichtigug rein, dass der Bauer durch die Queen beim Erstellen von s0  ersetzt wird, dies muss dazu noch rückgängi gemacht werden, wenn die Queen durch den Pawn ersetzt wird.
    global PIECE_LIST

    s0 = set()
    empty_board = chess.Board(None)
    empty_board.turn = turn
    tmp_piece_list = []
    s0.add(to_integer(empty_board, tmp_piece_list))
    
    total = 0
    not_matched = 0
   
    for piece in PIECE_LIST:
        s0, total, not_matched = fill_boards_with_piece(s0, piece, tmp_piece_list, total, not_matched)
        tmp_piece_list.append(piece)
        
    return total, not_matched

In [ ]:
def fill_boards_with_piece(int_boards_set, piece, cur_piece_list, total, not_matched):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(PIECE_LIST)
    for int_board in int_boards_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    continue

                if len(used_squares) + 1 < piece_count: #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
                    continue
                
                total += 1
                if board_int not in ALL_BOARDS:
                    if gaviota.probe_dtm(board_object) != 0: #0 is a draw and not thus relevant for this check
                        not_matched += 1
                    
    return finished_boards, total, not_matched


In [ ]:
def matchrate():
    total = 0
    not_matched = 0
    tmp_total, tmp_not_matched = matchrate_for_player(chess.WHITE)
    total += tmp_total
    not_matched += tmp_not_matched
    tmp_total, tmp_not_matched = matchrate_for_player(chess.BLACK)
    total += tmp_total
    not_matched += tmp_not_matched
    
    return total, not_matched

In [ ]:
PIECE_LIST, S_N_SEQ = load_data("s_n_knight_bishop_24_05")
S_N_SEQ = gen_all_integers(S_N_SEQ)
ALL_BOARDS = set.union(*S_N_SEQ)
total, not_matched = matchrate()
matching_rate = (total - not_matched)/total * 100
print("Matching Rate:" +  str(matching_rate) + "%")


In [ ]:
def compare_with_gaviota(gaviota, s_n_sequence):
    counter = 0
    
    for n in range(len(s_n_sequence)):
        count = 0
        for int in s_n_sequence[n]:
            chess_board = to_board(int, PIECE_LIST)
            if n != abs(gaviota.probe_dtm(chess_board)):
                count += 1
                print(chess_board)
        print(f"S{n}: Gaviota believes {count} of {len(s_n_sequence[n])} Situations are wrongly placed in the sequence.")
        counter += count
    print(f"Gaviota believes {counter} Situations are wrongly placed in the sequence.")
    return count


In [ ]:
PIECE_LIST, S_N_SEQ = load_data("s_n_two_bishop_24_05")
S_N_SEQ = gen_all_integers(S_N_SEQ)

gaviota = chess.gaviota.open_tablebase("./gaviota")


compare_with_gaviota(gaviota, S_N_SEQ)


In [ ]:
to_integer(chess.Board("8/8/8/8/8/B7/2K3B1/k7 w - - 0 1"), PIECE_LIST) in S_N_SEQ[5]

chess.Board("8/8/8/8/8/8/1BK3B1/k7 w - - 1 1").pseudo_legal_moves
